In [7]:
from ast import excepthandler
from operator import index
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import pandas as pd

In [8]:
# 라이브러리 import
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pandas as pd
import os

In [9]:
options = webdriver.ChromeOptions()
user_data = r"C:\Users\nemo2\AppData\Local\Google\Chrome\User Data"
options.add_argument(f'--user-data-dir={user_data}')
options.add_argument(f'profile-directory=Default')
#driver = webdriver.Chrome(options=options)

In [10]:
# 크롬 드라이버 불러오기
driver = webdriver.Chrome(options=options)

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x55bb7258c303 <unknown>
#1 0x55bb722712b7 <unknown>
#2 0x55bb722a5b40 <unknown>
#3 0x55bb722a16e5 <unknown>
#4 0x55bb722ebabc <unknown>
#5 0x55bb722df123 <unknown>
#6 0x55bb722af095 <unknown>
#7 0x55bb722b009e <unknown>
#8 0x55bb725506ab <unknown>
#9 0x55bb725544ba <unknown>
#10 0x55bb7253cf85 <unknown>
#11 0x55bb7255512f <unknown>
#12 0x55bb72520e6f <unknown>
#13 0x55bb725795f8 <unknown>
#14 0x55bb725797c2 <unknown>
#15 0x55bb7258b4a4 <unknown>
#16 0x7fdb1e541609 start_thread


In [11]:
# get url
#url = 'https://www.linkedin.com/login'
url = 'https://www.linkedin.com/'
driver.get(url)
time.sleep(2)

NameError: name 'driver' is not defined

In [6]:
keyword = 'RLHF'
URL = 'https://www.linkedin.com/search/results/people/?keywords=' + keyword
driver.get(url=URL)
driver.implicitly_wait(5)

In [7]:
# 수집할 프로필을 텍스트 파일로 저장하는 함수
def save_profile_to_txt(name, about, activity, experience, education, projects, skills, profile_url, page_number, cv_number):
    txt_filename = f"Page{page_number}_CV{cv_number}.txt"
    with open(txt_filename, "w", encoding="utf-8") as txt_file:
        txt_file.write(f"Name: {name}\n")
        txt_file.write(f"About: {about}\n")
        txt_file.write(f"Activity: {activity}\n")
        txt_file.write(f"Experience: {experience}\n")
        txt_file.write(f"Education: {education}\n")
        txt_file.write(f"Projects: {projects}\n")
        txt_file.write(f"Skills: {skills}\n")
        txt_file.write(f"Linked Link: {profile_url}\n")

In [8]:
# 프로필 정보를 가져와서 텍스트 파일로 저장하는 함수

# 방문한 프로필 URL을 저장할 집합 초기화
visited_profile_urls = set()

def get_text(driver, page_number, cv_number):
    profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
    if profile_links:
        for link in profile_links:
            profile_url = link.get_attribute('href')
            if profile_url not in visited_profile_urls:
                visited_profile_urls.add(profile_url)
                driver.get(profile_url)
                time.sleep(5)
                try:
                    name = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]').text.strip()
                    about = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]').text.strip()
                    activity = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]').text.strip()
                    experience = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]').text.strip()
                    education = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]').text.strip()
                    projects = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]').text.strip()
                    skills = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]').text.strip()
                    save_profile_to_txt(name, about, activity, experience, education, projects, skills, profile_url, page_number, cv_number)
                except Exception as e:
                    print(f"Error processing profile: {e}")
                finally:
                    driver.back()
                    time.sleep(2)

In [ ]:
with open('Page{page_number}_CV{cv_number}', 'w', newline = '') as file_output:
    haeders = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n', fieldnames= headers)
    writer.writeheader()
    
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        time.sleep(2)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        
        name_div = soup.find('div',{'class':'flex-1 mr5'})
        name_loc = name_div.find_all('ul')
        name = name_loc[0].find('li').get_text().strip()
        print('프로필 이름은: ', name)
        
        location = name_loc[1].find('li').get_text().strip()
        print('프로필 장소는: ', name)
        
        profile_title = name_div.find('h2').get_text().strip()
        print('job title: ', profile_title)
        print('\n')

In [9]:
# 끝까지 크롤링 되는 코드
# 페이지 수 만큼 반복하여 프로필 크롤링
for page_number in range(1, 6):
    cv_number = 1
    driver.get(f"https://www.linkedin.com/search/results/people/?keywords=RLHF&page={page_number}")  # 페이지로 이동
    time.sleep(5)  # 페이지 로딩 대기

    profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
    if profile_links:
        profile_urls = [link.get_attribute('href') for link in profile_links]
        for link_url in profile_urls:  
            try:
                if link_url not in visited_profile_urls:
                    visited_profile_urls.add(link_url)
                    driver.get(link_url)  # 프로필 링크로 이동
                    time.sleep(5)  # 프로필 로딩 대기
                    # 프로필 정보 수집
                    name_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]')
                    name = name_element.text.strip() if name_element else "N/A"
                    about_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]')
                    about = about_element.text.strip() if about_element else "N/A"
                    activity_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]')
                    activity = activity_element.text.strip() if activity_element else "N/A"
                    experience_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]')
                    experience = experience_element.text.strip() if experience_element else "N/A"
                    education_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]')
                    education = education_element.text.strip() if education_element else "N/A"
                    projects_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]')
                    projects = projects_element.text.strip() if projects_element else "N/A"
                    skills_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]')
                    skills = skills_element.text.strip() if skills_element else "N/A"
                    save_profile_to_txt(name, about, activity, experience, education, projects, skills, link_url, page_number, cv_number)
                    cv_number += 1
            except Exception as e:
                print(f"Error processing profile: {e}")
            finally:
                driver.back()  # 이전 페이지로 돌아가기
                time.sleep(2)
    else:
        print(f"No profile links found on page {page_number}")

Error processing profile: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7AB4C7012+3522402]
	(No symbol) [0x00007FF7AB0E8352]
	(No symbol) [0x00007FF7AAF95ABB]
	(No symbol) [0x00007FF7AAFDBF0E]
	(No symbol) [0x00007FF7AAFDC08C]
	(No symbol) [0x00007FF7AB01E437]
	(No symbol) [0x00007FF7AAFFF09F]
	(No symbol) [0x00007FF7AB01BDA3]
	(No symbol) [0x00007FF7AAFFEE03]
	(No symbol) [0x00007FF7AAFCF4D4]
	(No symbol) [0x00007FF7AAFD05F1]
	GetHandleVerifier [0x00007FF7AB4F9B9D+3730157]
	GetHandleVerifier [0x00007FF7AB54F02D+4079485]
	GetHandleVerifier [0x00007FF7AB5475D3+4048163]
	GetHandleVerifier [0x00007FF7AB21A649+718233]
	(No symbol) [0x00007FF7AB0F4A3F]
	(No symbol) [0

In [9]:
import time

def save_profile_to_txt(name, about, activity, experience, education, projects, skills, profile_url, page_number, cv_number):
    txt_filename = f"Page{page_number}_CV{cv_number}.txt"
    with open(txt_filename, "w", encoding="utf-8") as txt_file:
        txt_file.write(f"Name: {name}\n")
        txt_file.write(f"About: {about}\n")
        txt_file.write(f"Activity: {activity}\n")
        txt_file.write(f"Experience: {experience}\n")
        txt_file.write(f"Education: {education}\n")
        txt_file.write(f"Projects: {projects}\n")
        txt_file.write(f"Skills: {skills}\n")
        txt_file.write(f"Linked Link: {profile_url}\n")

def get_text(driver, page_number, cv_number):
    profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
    if profile_links:
        for link in profile_links:
            profile_url = link.get_attribute('href')
            if profile_url not in visited_profile_urls:
                visited_profile_urls.add(profile_url)
                driver.get(profile_url)
                time.sleep(5)
                try:
                    name = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]').text.strip()
                    about = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]').text.strip()
                    activity = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]').text.strip()
                    experience = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]').text.strip()
                    education = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]').text.strip()
                    projects = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]').text.strip()
                    skills = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]').text.strip()
                    save_profile_to_txt(name, about, activity, experience, education, projects, skills, profile_url, page_number, cv_number)
                except Exception as e:
                    print(f"Error processing profile: {e}")
                finally:
                    driver.back()
                    time.sleep(2)
# 방문한 프로필 URL을 저장할 집합 초기화
visited_profile_urls = set()

# 페이지의 첫 번째 프로필 수집
page_number = 1
cv_number = 1  # 프로필 번호 초기화

profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
if profile_links:
    for i in range(len(profile_links)):
        profile_link = profile_links[i].get_attribute('href')
        driver.get(profile_link)
        time.sleep(5)  # 페이지가 로드될 때까지 충분한 시간 대기
        get_text(driver, page_number, cv_number)
        cv_number += 1  # 프로필 번호 증가
else:
    print("No profile links found")

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7AB4C7012+3522402]
	(No symbol) [0x00007FF7AB0E8352]
	(No symbol) [0x00007FF7AAF95ABB]
	(No symbol) [0x00007FF7AAFA50FB]
	(No symbol) [0x00007FF7AAF9B8F7]
	(No symbol) [0x00007FF7AAF9BA3F]
	(No symbol) [0x00007FF7AAF99CE0]
	(No symbol) [0x00007FF7AAF9D42A]
	(No symbol) [0x00007FF7AB01CA1B]
	(No symbol) [0x00007FF7AAFFF05A]
	(No symbol) [0x00007FF7AB01BDA3]
	(No symbol) [0x00007FF7AAFFEE03]
	(No symbol) [0x00007FF7AAFCF4D4]
	(No symbol) [0x00007FF7AAFD05F1]
	GetHandleVerifier [0x00007FF7AB4F9B9D+3730157]
	GetHandleVerifier [0x00007FF7AB54F02D+4079485]
	GetHandleVerifier [0x00007FF7AB5475D3+4048163]
	GetHandleVerifier [0x00007FF7AB21A649+718233]
	(No symbol) [0x00007FF

In [7]:
# llm 키워드로 검색한 페이지로 이동
def go_to_search_page(keyword):
    URL = 'https://www.linkedin.com/search/results/people/?keywords=' + keyword
    driver.get(url=URL)
    time.sleep(5)

# llm 키워드로 검색한 페이지로 이동
go_to_search_page('RLHF')

In [8]:
# 수집할 프로필을 텍스트 파일로 저장하는 함수
def save_profile_to_txt(name, about, activity, experience, education, projects, skills, profile_url, page_number, cv_number):
    txt_filename = f"CV_Page{page_number}_CV{cv_number}.txt"
    with open(txt_filename, "w", encoding="utf-8") as txt_file:
        txt_file.write(f"Name: {name}\n")
        txt_file.write(f"About: {about}\n")
        txt_file.write(f"Activity: {activity}\n")
        txt_file.write(f"Experience: {experience}\n")
        txt_file.write(f"Education: {education}\n")
        txt_file.write(f"Projects: {projects}\n")
        txt_file.write(f"Skills: {skills}\n")
        txt_file.write(f"Linked Link: {profile_url}\n")

In [10]:
# 프로필 정보를 가져와서 텍스트 파일로 저장하는 함수

# 방문한 프로필 URL을 저장할 집합 초기화
visited_profile_urls = set()

def get_text(driver, page_number, cv_number):
    profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
    if profile_links:
        for link in profile_links:
            profile_url = link.get_attribute('href')
            if profile_url not in visited_profile_urls:
                visited_profile_urls.add(profile_url)
                driver.get(profile_url)
                time.sleep(5)
                try:
                    name = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]').text.strip()
                    about = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]').text.strip()
                    activity = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]').text.strip()
                    experience = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]').text.strip()
                    education = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]').text.strip()
                    projects = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]').text.strip()
                    skills = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]').text.strip()
                    save_profile_to_txt(name, about, activity, experience, education, projects, skills, profile_url, page_number, cv_number)
                except Exception as e:
                    print(f"Error processing profile: {e}")
                finally:
                    driver.back()
                    time.sleep(2)

In [11]:
# 페이지 수 만큼 반복하여 프로필 크롤링
for page_number in range(1, 11):
    cv_number = 1
    driver.get(f"https://www.linkedin.com/search/results/people/?keywords=RLHF&page={page_number}")  # 페이지로 이동
    time.sleep(5)  # 페이지 로딩 대기

    profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
    if profile_links:
        profile_urls = [link.get_attribute('href') for link in profile_links]
        for link_url in profile_urls:  
            try:
                if link_url not in visited_profile_urls:
                    visited_profile_urls.add(link_url)
                    driver.get(link_url)  # 프로필 링크로 이동
                    time.sleep(5)  # 프로필 로딩 대기
                    # 프로필 정보 수집
                    name_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]')
                    name = name_element.text.strip() if name_element else "N/A"
                    about_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]')
                    about = about_element.text.strip() if about_element else "N/A"
                    activity_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]')
                    activity = activity_element.text.strip() if activity_element else "N/A"
                    experience_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]')
                    experience = experience_element.text.strip() if experience_element else "N/A"
                    education_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]')
                    education = education_element.text.strip() if education_element else "N/A"
                    projects_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]')
                    projects = projects_element.text.strip() if projects_element else "N/A"
                    skills_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]')
                    skills = skills_element.text.strip() if skills_element else "N/A"
                    save_profile_to_txt(name, about, activity, experience, education, projects, skills, link_url, page_number, cv_number)
                    cv_number += 1
            except Exception as e:
                print(f"Error processing profile: {e}")
            finally:
                driver.back()  # 이전 페이지로 돌아가기
                time.sleep(2)
    else:
        print(f"No profile links found on page {page_number}")


Error processing profile: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]"}
  (Session info: chrome=121.0.6167.161); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CB935E42+3538674]
	(No symbol) [0x00007FF7CB554C02]
	(No symbol) [0x00007FF7CB405AEB]
	(No symbol) [0x00007FF7CB44BF4E]
	(No symbol) [0x00007FF7CB44C0CC]
	(No symbol) [0x00007FF7CB48E477]
	(No symbol) [0x00007FF7CB46F0EF]
	(No symbol) [0x00007FF7CB48BDE3]
	(No symbol) [0x00007FF7CB46EE53]
	(No symbol) [0x00007FF7CB43F514]
	(No symbol) [0x00007FF7CB440631]
	GetHandleVerifier [0x00007FF7CB966CAD+3738973]
	GetHandleVerifier [0x00007FF7CB9BC506+4089270]
	GetHandleVerifier [0x00007FF7CB9B4823+4057299]
	GetHandleVerifier [0x00007FF7CB685C49+720121]
	(No symbol) [0x00007FF7CB56126F]
	(No symbol) [0

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.161)
Stacktrace:
	GetHandleVerifier [0x00007FF7CB935E42+3538674]
	(No symbol) [0x00007FF7CB554C02]
	(No symbol) [0x00007FF7CB405AEB]
	(No symbol) [0x00007FF7CB3E288C]
	(No symbol) [0x00007FF7CB475DD7]
	(No symbol) [0x00007FF7CB48B40F]
	(No symbol) [0x00007FF7CB46EE53]
	(No symbol) [0x00007FF7CB43F514]
	(No symbol) [0x00007FF7CB440631]
	GetHandleVerifier [0x00007FF7CB966CAD+3738973]
	GetHandleVerifier [0x00007FF7CB9BC506+4089270]
	GetHandleVerifier [0x00007FF7CB9B4823+4057299]
	GetHandleVerifier [0x00007FF7CB685C49+720121]
	(No symbol) [0x00007FF7CB56126F]
	(No symbol) [0x00007FF7CB55C304]
	(No symbol) [0x00007FF7CB55C432]
	(No symbol) [0x00007FF7CB54BD04]
	BaseThreadInitThunk [0x00007FFAE586257D+29]
	RtlUserThreadStart [0x00007FFAE74CAA58+40]


In [10]:
# 페이지 수 만큼 반복하여 프로필 크롤링
for page_number in range(1, 11):
    cv_number = 1
    driver.get(f"https://www.linkedin.com/search/results/people/?keywords=RLHF&page={page_number}")  # 페이지로 이동
    time.sleep(5)  # 페이지 로딩 대기

    profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
    if profile_links:
        for link in profile_links:  
            try:
                link_url = link.get_attribute('href')  # 프로필 링크 URL 가져오기
                if link_url not in visited_profile_urls:
                    visited_profile_urls.add(link_url)
                    link.click()  # 프로필 링크 클릭
                    time.sleep(5)  # 프로필 로딩 대기
                    # 프로필 정보 수집
                    name_xpath = link.find_element(By.XPATH, "../../../../../../../../../../../../../../../..//span[contains(@class,'name')]/span/a")
                    name = name_xpath.text.strip() if name_xpath else "N/A"
                    if name != "LinkedIn Member":  # LinkedIn Member인 경우 제외
                        about_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]')
                        about = about_element.text.strip() if about_element else "N/A"
                        activity_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]')
                        activity = activity_element.text.strip() if activity_element else "N/A"
                        experience_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]')
                        experience = experience_element.text.strip() if experience_element else "N/A"
                        education_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]')
                        education = education_element.text.strip() if education_element else "N/A"
                        projects_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]')
                        projects = projects_element.text.strip() if projects_element else "N/A"
                        skills_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]')
                        skills = skills_element.text.strip() if skills_element else "N/A"
                        save_profile_to_txt(name, about, activity, experience, education, projects, skills, link_url, page_number, cv_number)
                        cv_number += 1
            except Exception as e:
                print(f"Error processing profile: {e}")
            finally:
                driver.back()  # 이전 페이지로 돌아가기
                time.sleep(2)
                    
    else:
        print(f"No profile links found on page {page_number}")


Error processing profile: Message: stale element reference: stale element not found
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7AB4C7012+3522402]
	(No symbol) [0x00007FF7AB0E8352]
	(No symbol) [0x00007FF7AAF95ABB]
	(No symbol) [0x00007FF7AAF9AD09]
	(No symbol) [0x00007FF7AAF9D110]
	(No symbol) [0x00007FF7AAF9D1D0]
	(No symbol) [0x00007FF7AAFDBB7C]
	(No symbol) [0x00007FF7AAFDC08C]
	(No symbol) [0x00007FF7AAFD12AC]
	(No symbol) [0x00007FF7AAFFF09F]
	(No symbol) [0x00007FF7AAFD120A]
	(No symbol) [0x00007FF7AAFFF270]
	(No symbol) [0x00007FF7AB01BDA3]
	(No symbol) [0x00007FF7AAFFEE03]
	(No symbol) [0x00007FF7AAFCF4D4]
	(No symbol) [0x00007FF7AAFD05F1]
	GetHandleVerifier [0x00007FF7AB4F9B9D+3730157]
	GetHandleVerifier [0x00007FF7AB54F02D+4079485]
	GetHandleVerifier [0x00007FF7AB5475D3+4048163]
	GetHand

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.185)
Stacktrace:
	GetHandleVerifier [0x00007FF7AB4C7012+3522402]
	(No symbol) [0x00007FF7AB0E8352]
	(No symbol) [0x00007FF7AAF95ABB]
	(No symbol) [0x00007FF7AAF7287C]
	(No symbol) [0x00007FF7AB005D97]
	(No symbol) [0x00007FF7AB01B3CF]
	(No symbol) [0x00007FF7AAFFEE03]
	(No symbol) [0x00007FF7AAFCF4D4]
	(No symbol) [0x00007FF7AAFD05F1]
	GetHandleVerifier [0x00007FF7AB4F9B9D+3730157]
	GetHandleVerifier [0x00007FF7AB54F02D+4079485]
	GetHandleVerifier [0x00007FF7AB5475D3+4048163]
	GetHandleVerifier [0x00007FF7AB21A649+718233]
	(No symbol) [0x00007FF7AB0F4A3F]
	(No symbol) [0x00007FF7AB0EFA94]
	(No symbol) [0x00007FF7AB0EFBC2]
	(No symbol) [0x00007FF7AB0DF2E4]
	BaseThreadInitThunk [0x00007FFD122F257D+29]
	RtlUserThreadStart [0x00007FFD1286AA58+40]


In [9]:
# 페이지 수 만큼 반복하여 프로필 크롤링
for page_number in range(1, 5):
    cv_number = 1
    driver.get(f"https://www.linkedin.com/search/results/people/?keywords=RLHF&page={page_number}")  # 페이지로 이동
    time.sleep(5)  # 페이지 로딩 대기

    while True:
        # 상단에 있는 프로필 정보 수집
        profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
        if profile_links:
            for link in profile_links:  
                try:
                    link_url = link.get_attribute('href')  # 프로필 링크 URL 가져오기
                    if link_url not in visited_profile_urls:
                        visited_profile_urls.add(link_url)
                        link.click()  # 프로필 링크 클릭
                        time.sleep(5)  # 프로필 로딩 대기
                        # 프로필 정보 수집
                        name_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]')
                        name = name_element.text.strip() if name_element else "N/A"
                        about_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]')
                        about = about_element.text.strip() if about_element else "N/A"
                        activity_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]')
                        activity = activity_element.text.strip() if activity_element else "N/A"
                        experience_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]')
                        experience = experience_element.text.strip() if experience_element else "N/A"
                        education_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]')
                        education = education_element.text.strip() if education_element else "N/A"
                        projects_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]')
                        projects = projects_element.text.strip() if projects_element else "N/A"
                        skills_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]')
                        skills = skills_element.text.strip() if skills_element else "N/A"
                        save_profile_to_txt(name, about, activity, experience, education, projects, skills, link_url, page_number, cv_number)
                        cv_number += 1
                except Exception as e:
                    print(f"Error processing profile: {e}")
                finally:
                    driver.back()  # 이전 페이지로 돌아가기
                    time.sleep(2)
        else:
            print(f"No profile links found on page {page_number}")
        
        # 페이지 하단으로 스크롤하여 추가 프로필 로드
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.4);")
        time.sleep(5)  # 스크롤 로딩 대기
        
        # 추가로 로드된 프로필 수집
        additional_profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
        for additional_link in additional_profile_links[len(profile_links):]:
            try:
                link_url = additional_link.get_attribute('href')  # 프로필 링크 URL 가져오기
                if link_url not in visited_profile_urls:
                    visited_profile_urls.add(link_url)
                    additional_link.click()  # 프로필 링크 클릭
                    time.sleep(5)  # 프로필 로딩 대기
                    # 프로필 정보 수집
                    name_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]')
                    name = name_element.text.strip() if name_element else "N/A"
                    about_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]')
                    about = about_element.text.strip() if about_element else "N/A"
                    activity_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]')
                    activity = activity_element.text.strip() if activity_element else "N/A"
                    experience_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]')
                    experience = experience_element.text.strip() if experience_element else "N/A"
                    education_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]')
                    education = education_element.text.strip() if education_element else "N/A"
                    projects_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]')
                    projects = projects_element.text.strip() if projects_element else "N/A"
                    skills_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]')
                    skills = skills_element.text.strip() if skills_element else "N/A"
                    save_profile_to_txt(name, about, activity, experience, education, projects, skills, link_url, page_number, cv_number)
                    cv_number += 1
            except Exception as e:
                print(f"Error processing profile: {e}")
            finally:
                driver.back()  # 이전 페이지로 돌아가기
                time.sleep(2)
        
        # 더 이상 페이지 하단에 프로필이 로드되지 않을 때까지 반복
        if len(additional_profile_links) == len(profile_links):
            break


Error processing profile: Message: stale element reference: stale element not found
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7AB4C7012+3522402]
	(No symbol) [0x00007FF7AB0E8352]
	(No symbol) [0x00007FF7AAF95ABB]
	(No symbol) [0x00007FF7AAF9AD09]
	(No symbol) [0x00007FF7AAF9D42A]
	(No symbol) [0x00007FF7AB01CA1B]
	(No symbol) [0x00007FF7AAFFF05A]
	(No symbol) [0x00007FF7AB01BDA3]
	(No symbol) [0x00007FF7AAFFEE03]
	(No symbol) [0x00007FF7AAFCF4D4]
	(No symbol) [0x00007FF7AAFD05F1]
	GetHandleVerifier [0x00007FF7AB4F9B9D+3730157]
	GetHandleVerifier [0x00007FF7AB54F02D+4079485]
	GetHandleVerifier [0x00007FF7AB5475D3+4048163]
	GetHandleVerifier [0x00007FF7AB21A649+718233]
	(No symbol) [0x00007FF7AB0F4A3F]
	(No symbol) [0x00007FF7AB0EFA94]
	(No symbol) [0x00007FF7AB0EFBC2]
	(No symbol) [0x00007FF7AB0DF

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.185)
Stacktrace:
	GetHandleVerifier [0x00007FF7AB4C7012+3522402]
	(No symbol) [0x00007FF7AB0E8352]
	(No symbol) [0x00007FF7AAF95ABB]
	(No symbol) [0x00007FF7AAF7287C]
	(No symbol) [0x00007FF7AB005D97]
	(No symbol) [0x00007FF7AB01B3CF]
	(No symbol) [0x00007FF7AAFFEE03]
	(No symbol) [0x00007FF7AAFCF4D4]
	(No symbol) [0x00007FF7AAFD05F1]
	GetHandleVerifier [0x00007FF7AB4F9B9D+3730157]
	GetHandleVerifier [0x00007FF7AB54F02D+4079485]
	GetHandleVerifier [0x00007FF7AB5475D3+4048163]
	GetHandleVerifier [0x00007FF7AB21A649+718233]
	(No symbol) [0x00007FF7AB0F4A3F]
	(No symbol) [0x00007FF7AB0EFA94]
	(No symbol) [0x00007FF7AB0EFBC2]
	(No symbol) [0x00007FF7AB0DF2E4]
	BaseThreadInitThunk [0x00007FFD122F257D+29]
	RtlUserThreadStart [0x00007FFD1286AA58+40]


In [18]:
# 페이지 수 만큼 반복하여 프로필 크롤링
for page_number in range(1, 21):
    cv_number = 1
    driver.get(f"https://www.linkedin.com/search/results/people/?keywords=RLHF&page={page_number}")  # 페이지로 이동
    time.sleep(5)  # 페이지 로딩 대기

    profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
    if profile_links:
        for link in profile_links:  
            try:
                link_url = link.get_attribute('href')  # 프로필 링크 URL 가져오기
                if link_url not in visited_profile_urls:
                    visited_profile_urls.add(link_url)
                    link.click()  # 프로필 링크 클릭
                    time.sleep(5)  # 프로필 로딩 대기
                    # 프로필 정보 수집
                    name_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]')
                    name = name_element.text.strip() if name_element else "N/A"
                    about_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]')
                    about = about_element.text.strip() if about_element else "N/A"
                    activity_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]')
                    activity = activity_element.text.strip() if activity_element else "N/A"
                    experience_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]')
                    experience = experience_element.text.strip() if experience_element else "N/A"
                    education_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]')
                    education = education_element.text.strip() if education_element else "N/A"
                    projects_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]')
                    projects = projects_element.text.strip() if projects_element else "N/A"
                    skills_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]')
                    skills = skills_element.text.strip() if skills_element else "N/A"
                    save_profile_to_txt(name, about, activity, experience, education, projects, skills, link_url, page_number, cv_number)
                    cv_number += 1
            except Exception as e:
                print(f"Error processing profile: {e}")
            finally:
                driver.back()  # 이전 페이지로 돌아가기
                time.sleep(2)
                    
    else:
        print(f"No profile links found on page {page_number}")


Error processing profile: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]"}
  (Session info: chrome=121.0.6167.161); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CB935E42+3538674]
	(No symbol) [0x00007FF7CB554C02]
	(No symbol) [0x00007FF7CB405AEB]
	(No symbol) [0x00007FF7CB44BF4E]
	(No symbol) [0x00007FF7CB44C0CC]
	(No symbol) [0x00007FF7CB48E477]
	(No symbol) [0x00007FF7CB46F0EF]
	(No symbol) [0x00007FF7CB48BDE3]
	(No symbol) [0x00007FF7CB46EE53]
	(No symbol) [0x00007FF7CB43F514]
	(No symbol) [0x00007FF7CB440631]
	GetHandleVerifier [0x00007FF7CB966CAD+3738973]
	GetHandleVerifier [0x00007FF7CB9BC506+4089270]
	GetHandleVerifier [0x00007FF7CB9B4823+4057299]
	GetHandleVerifier [0x00007FF7CB685C49+720121]
	(No symbol) [0x00007FF7CB56126F]
	(No symbol) [0

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.161)
Stacktrace:
	GetHandleVerifier [0x00007FF7CB935E42+3538674]
	(No symbol) [0x00007FF7CB554C02]
	(No symbol) [0x00007FF7CB405AEB]
	(No symbol) [0x00007FF7CB3E288C]
	(No symbol) [0x00007FF7CB475DD7]
	(No symbol) [0x00007FF7CB48B40F]
	(No symbol) [0x00007FF7CB46EE53]
	(No symbol) [0x00007FF7CB43F514]
	(No symbol) [0x00007FF7CB440631]
	GetHandleVerifier [0x00007FF7CB966CAD+3738973]
	GetHandleVerifier [0x00007FF7CB9BC506+4089270]
	GetHandleVerifier [0x00007FF7CB9B4823+4057299]
	GetHandleVerifier [0x00007FF7CB685C49+720121]
	(No symbol) [0x00007FF7CB56126F]
	(No symbol) [0x00007FF7CB55C304]
	(No symbol) [0x00007FF7CB55C432]
	(No symbol) [0x00007FF7CB54BD04]
	BaseThreadInitThunk [0x00007FFAE586257D+29]
	RtlUserThreadStart [0x00007FFAE74CAA58+40]


In [12]:
# 페이지 수 만큼 반복하여 프로필 크롤링
for page_number in range(1, 11):
    cv_number = 1
    driver.get(f"https://www.linkedin.com/search/results/people/?keywords=RLHF&page={page_number}")  # 페이지로 이동
    time.sleep(5)  # 페이지 로딩 대기

    profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
    if profile_links:
        for link in profile_links:  
            try:
                link_url = link.get_attribute('href')  # 프로필 링크 URL 가져오기
                if link_url not in visited_profile_urls:
                    visited_profile_urls.add(link_url)
                    link.click()  # 프로필 링크 클릭
                    time.sleep(5)  # 프로필 로딩 대기
                    # 프로필 정보 수집
                    name = driver.find_element(By.XPATH, '//h1').text.strip()
                    about = driver.find_element(By.XPATH, '//section[contains(@class, "about")]').text.strip()
                    activity = driver.find_element(By.XPATH, '//section[contains(@class, "activity")]').text.strip()
                    experience = driver.find_element(By.XPATH, '//section[contains(@class, "experience")]').text.strip()
                    education = driver.find_element(By.XPATH, '//section[contains(@class, "education")]').text.strip()
                    projects = driver.find_element(By.XPATH, '//section[contains(@class, "projects")]').text.strip()
                    skills = driver.find_element(By.XPATH, '//section[contains(@class, "skills")]').text.strip()
                    save_profile_to_txt(name, about, activity, experience, education, projects, skills, link_url, page_number, cv_number)
                    cv_number += 1
            except Exception as e:
                print(f"Error processing profile: {e}")
            finally:
                driver.back()  # 이전 페이지로 돌아가기
                time.sleep(2)
                    
    else:
        print(f"No profile links found on page {page_number}")


Error processing profile: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//section[contains(@class, "about")]"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7AB4C7012+3522402]
	(No symbol) [0x00007FF7AB0E8352]
	(No symbol) [0x00007FF7AAF95ABB]
	(No symbol) [0x00007FF7AAFDBF0E]
	(No symbol) [0x00007FF7AAFDC08C]
	(No symbol) [0x00007FF7AB01E437]
	(No symbol) [0x00007FF7AAFFF09F]
	(No symbol) [0x00007FF7AB01BDA3]
	(No symbol) [0x00007FF7AAFFEE03]
	(No symbol) [0x00007FF7AAFCF4D4]
	(No symbol) [0x00007FF7AAFD05F1]
	GetHandleVerifier [0x00007FF7AB4F9B9D+3730157]
	GetHandleVerifier [0x00007FF7AB54F02D+4079485]
	GetHandleVerifier [0x00007FF7AB5475D3+4048163]
	GetHandleVerifier [0x00007FF7AB21A649+718233]
	(No symbol) [0x00007FF7AB0F4A3F]
	(No symbol) [0x00007FF7AB0EFA94]
	(No sym

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.185)
Stacktrace:
	GetHandleVerifier [0x00007FF7AB4C7012+3522402]
	(No symbol) [0x00007FF7AB0E8352]
	(No symbol) [0x00007FF7AAF95ABB]
	(No symbol) [0x00007FF7AAF7287C]
	(No symbol) [0x00007FF7AB005D97]
	(No symbol) [0x00007FF7AB01B3CF]
	(No symbol) [0x00007FF7AAFFEE03]
	(No symbol) [0x00007FF7AAFCF4D4]
	(No symbol) [0x00007FF7AAFD05F1]
	GetHandleVerifier [0x00007FF7AB4F9B9D+3730157]
	GetHandleVerifier [0x00007FF7AB54F02D+4079485]
	GetHandleVerifier [0x00007FF7AB5475D3+4048163]
	GetHandleVerifier [0x00007FF7AB21A649+718233]
	(No symbol) [0x00007FF7AB0F4A3F]
	(No symbol) [0x00007FF7AB0EFA94]
	(No symbol) [0x00007FF7AB0EFBC2]
	(No symbol) [0x00007FF7AB0DF2E4]
	BaseThreadInitThunk [0x00007FFD122F257D+29]
	RtlUserThreadStart [0x00007FFD1286AA58+40]


In [73]:
# 페이지 수 만큼 반복하여 프로필 크롤링
for page_number in range(1, 11):
    cv_number = 1
    driver.get(f"https://www.linkedin.com/search/results/people/?keywords=RLHF&page={page_number}")  # 페이지로 이동
    time.sleep(5)  # 페이지 로딩 대기

    profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
    if profile_links:
        for link in profile_links:  
            try:
                link.click()  # 프로필 링크 클릭
                time.sleep(5)  # 프로필 로딩 대기
                
                # WebElement가 유효한지 확인
                if link != driver.switch_to.active_element:
                    continue
                
                profile_url = driver.current_url  # 현재 페이지의 URL 가져오기
                if profile_url not in visited_profile_urls:
                    visited_profile_urls.add(profile_url)
                    name = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]').text.strip()
                    about = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]').text.strip()
                    activity = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]').text.strip()
                    experience = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]').text.strip()
                    education = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]').text.strip()
                    projects = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]').text.strip()
                    skills = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]').text.strip()
                    save_profile_to_txt(name, about, activity, experience, education, projects, skills, profile_url, page_number, cv_number)
            except Exception as e:
                print(f"Error processing profile: {e}")
            finally:
                driver.back()  # 이전 페이지로 돌아가기
                time.sleep(2)
                    
    else:
        print(f"No profile links found on page {page_number}")


Error processing profile: Message: stale element reference: stale element not found
  (Session info: chrome=121.0.6167.161); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CB935E42+3538674]
	(No symbol) [0x00007FF7CB554C02]
	(No symbol) [0x00007FF7CB405AEB]
	(No symbol) [0x00007FF7CB40AD39]
	(No symbol) [0x00007FF7CB40D150]
	(No symbol) [0x00007FF7CB40D210]
	(No symbol) [0x00007FF7CB44DED5]
	(No symbol) [0x00007FF7CB4416D6]
	(No symbol) [0x00007FF7CB46F0AA]
	(No symbol) [0x00007FF7CB44124A]
	(No symbol) [0x00007FF7CB46F2C0]
	(No symbol) [0x00007FF7CB48BDE3]
	(No symbol) [0x00007FF7CB46EE53]
	(No symbol) [0x00007FF7CB43F514]
	(No symbol) [0x00007FF7CB440631]
	GetHandleVerifier [0x00007FF7CB966CAD+3738973]
	GetHandleVerifier [0x00007FF7CB9BC506+4089270]
	GetHandleVerifier [0x00007FF7CB9B4823+4057299]
	GetHandleVerifier [0x00007FF7CB685C49+720

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.161)
Stacktrace:
	GetHandleVerifier [0x00007FF7CB935E42+3538674]
	(No symbol) [0x00007FF7CB554C02]
	(No symbol) [0x00007FF7CB405AEB]
	(No symbol) [0x00007FF7CB3E288C]
	(No symbol) [0x00007FF7CB475DD7]
	(No symbol) [0x00007FF7CB48B40F]
	(No symbol) [0x00007FF7CB46EE53]
	(No symbol) [0x00007FF7CB43F514]
	(No symbol) [0x00007FF7CB440631]
	GetHandleVerifier [0x00007FF7CB966CAD+3738973]
	GetHandleVerifier [0x00007FF7CB9BC506+4089270]
	GetHandleVerifier [0x00007FF7CB9B4823+4057299]
	GetHandleVerifier [0x00007FF7CB685C49+720121]
	(No symbol) [0x00007FF7CB56126F]
	(No symbol) [0x00007FF7CB55C304]
	(No symbol) [0x00007FF7CB55C432]
	(No symbol) [0x00007FF7CB54BD04]
	BaseThreadInitThunk [0x00007FFAE586257D+29]
	RtlUserThreadStart [0x00007FFAE74CAA58+40]


In [45]:
# 페이지 수 만큼 반복하여 프로필 크롤링
for page_number in range(1, 11):
    cv_number = 1
    while True:
        get_text(driver, page_number, cv_number)
        cv_number += 1
        next_button = driver.find_element(By.XPATH, f'//button[contains(text(), "{page_number+1}")]')
        if next_button.get_attribute('disabled'):
            break
        next_button.click()
        time.sleep(5)  # 다음 페이지가 로드될 때까지 기다리기
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") # 페이지 하단으로 스크롤
        time.sleep(2)

Error processing profile: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]"}
  (Session info: chrome=121.0.6167.161); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CB935E42+3538674]
	(No symbol) [0x00007FF7CB554C02]
	(No symbol) [0x00007FF7CB405AEB]
	(No symbol) [0x00007FF7CB44BF4E]
	(No symbol) [0x00007FF7CB44C0CC]
	(No symbol) [0x00007FF7CB48E477]
	(No symbol) [0x00007FF7CB46F0EF]
	(No symbol) [0x00007FF7CB48BDE3]
	(No symbol) [0x00007FF7CB46EE53]
	(No symbol) [0x00007FF7CB43F514]
	(No symbol) [0x00007FF7CB440631]
	GetHandleVerifier [0x00007FF7CB966CAD+3738973]
	GetHandleVerifier [0x00007FF7CB9BC506+4089270]
	GetHandleVerifier [0x00007FF7CB9B4823+4057299]
	GetHandleVerifier [0x00007FF7CB685C49+720121]
	(No symbol) [0x00007FF7CB56126F]
	(No symbol) [0

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=121.0.6167.161); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CB935E42+3538674]
	(No symbol) [0x00007FF7CB554C02]
	(No symbol) [0x00007FF7CB405AEB]
	(No symbol) [0x00007FF7CB41514B]
	(No symbol) [0x00007FF7CB40B92A]
	(No symbol) [0x00007FF7CB40BA76]
	(No symbol) [0x00007FF7CB409D10]
	(No symbol) [0x00007FF7CB40D46A]
	(No symbol) [0x00007FF7CB48CA5B]
	(No symbol) [0x00007FF7CB46F0AA]
	(No symbol) [0x00007FF7CB48BDE3]
	(No symbol) [0x00007FF7CB46EE53]
	(No symbol) [0x00007FF7CB43F514]
	(No symbol) [0x00007FF7CB440631]
	GetHandleVerifier [0x00007FF7CB966CAD+3738973]
	GetHandleVerifier [0x00007FF7CB9BC506+4089270]
	GetHandleVerifier [0x00007FF7CB9B4823+4057299]
	GetHandleVerifier [0x00007FF7CB685C49+720121]
	(No symbol) [0x00007FF

In [8]:
# 수집할 프로필을 텍스트 파일로 저장하는 함수
def save_profile_to_txt(name, about, activity, experience, education, projects, skills, profile_url, page_number, cv_number):
    txt_filename = f"CVcv_{cv_number}.txt"
    with open(txt_filename, "w", encoding="utf-8") as txt_file:
        txt_file.write(f"Name: {name}\n")
        txt_file.write(f"About: {about}\n")
        txt_file.write(f"Activity: {activity}\n")
        txt_file.write(f"Experience: {experience}\n")
        txt_file.write(f"Education: {education}\n")
        txt_file.write(f"Projects: {projects}\n")
        txt_file.write(f"Skills: {skills}\n")
        txt_file.write(f"Linked Link: {profile_url}\n")

# 프로필 정보를 가져와서 텍스트 파일로 저장하는 함수
def get_text(driver,cv_number):
    # 이름
    try:
        name = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]').text.strip()
    except:
        name = ""

    # About
    try:
        about = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]').text.strip()
    except:
        about = ""

    # Activity
    try:
        activity = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]').text.strip()
    except:
        activity = ""

    # Experience
    try:
        experience = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]').text.strip()
    except:
        experience = ""

    # Education
    try:
        education = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]').text.strip()
    except:
        education = ""

    # Projects
    try:
        projects = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]').text.strip()
    except:
        projects = ""

    # Skills
    try:
        skills = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]').text.strip()
    except:
        skills = ""
    
    # 프로필 URL 가져오기
    profile_url = driver.current_url
    
    # 중복 방문을 체크하고 방문한 프로필은 무시함
    if profile_url not in visited_profile_urls:
        # 정보 저장
        save_profile_to_txt(name, about, activity, experience, education, projects, skills, profile_url, page_number, cv_number)
        visited_profile_urls.add(profile_url)  # 방문한 프로필 URL 추가


# 방문한 프로필 URL을 저장할 집합 초기화
visited_profile_urls = set()


In [8]:
from selenium.common.exceptions import StaleElementReferenceException

In [12]:
visited_profile_urls = set()  # 이미 방문한 프로필 링크를 추적하기 위한 세트

def collect_profiles_on_page(driver, cv_number):
    profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
    if profile_links:
        for link in profile_links:
            try:
                link_url = link.get_attribute('href')
                if link_url not in visited_profile_urls:
                    visited_profile_urls.add(link_url)
                    link.click()  # 프로필 링크 클릭
                    time.sleep(5)  # 프로필 로딩 대기
                    # 프로필 정보 수집
                    name_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]')
                    name = name_element.text.strip() if name_element else "N/A"
                    about_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]')
                    about = about_element.text.strip() if about_element else "N/A"
                    activity_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]')
                    activity = activity_element.text.strip() if activity_element else "N/A"
                    experience_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]')
                    experience = experience_element.text.strip() if experience_element else "N/A"
                    education_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]')
                    education = education_element.text.strip() if education_element else "N/A"
                    projects_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]')
                    projects = projects_element.text.strip() if projects_element else "N/A"
                    skills_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]')
                    skills = skills_element.text.strip() if skills_element else "N/A"
                    save_profile_to_txt(name, about, activity, experience, education, projects, skills, link_url, page_number, cv_number)
                    cv_number += 1
                    driver.back()  # 이전 페이지로 돌아가기
                    time.sleep(2)  # 페이지가 완전히 로드될 때까지 대기
                else:
                    print(f"Profile {link_url} already visited")
            except StaleElementReferenceException:
                print("StaleElementReferenceException occurred. Skipping the current profile...")
                continue
    else:
        print("No profile links found on this page")


cv_number = 1  # 프로필 번호 초기화
for page_number in range(1, 6):  
    driver.get(f"https://www.linkedin.com/search/results/people/?keywords=RLHF&page={page_number}")  # 페이지로 이동
    time.sleep(5)  # 페이지 로딩 대기

    collect_profiles_on_page(driver, cv_number)

    # 페이지 이동 확인
    if page_number % 20 == 0:  # 한 페이지에서 일정 수의 프로필을 수집한 후 다음 페이지로 이동
        next_button_xpath = f'/html/body/div[4]/div[3]/div[2]/div/div[1]/main/div/div/div[4]/div/div/ul/li[{(page_number // 20) + 2}]/button'
        next_button = driver.find_element(By.XPATH, next_button_xpath)
        next_button.click()
        time.sleep(5)  # 페이지 이동 대기


StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping the current profile...
StaleElementReferenceException occurred. Skipping t

: 

In [9]:
visited_profile_urls = set()  # 이미 방문한 프로필 링크를 추적하기 위한 세트

def collect_profiles_on_page(driver, cv_number):
    profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
    if profile_links:
        for link in profile_links:
            try:
                link_url = link.get_attribute('href')
                if link_url not in visited_profile_urls:
                    visited_profile_urls.add(link_url)
                    link.click()  # 프로필 링크 클릭
                    time.sleep(5)  # 프로필 로딩 대기
                    # 프로필 정보 수집
                    name_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]')
                    name = name_element.text.strip() if name_element else "N/A"
                    about_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[2]')
                    about = about_element.text.strip() if about_element else "N/A"
                    activity_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[3]')
                    activity = activity_element.text.strip() if activity_element else "N/A"
                    experience_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]')
                    experience = experience_element.text.strip() if experience_element else "N/A"
                    education_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]')
                    education = education_element.text.strip() if education_element else "N/A"
                    projects_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]')
                    projects = projects_element.text.strip() if projects_element else "N/A"
                    skills_element = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[7]')
                    skills = skills_element.text.strip() if skills_element else "N/A"
                    save_profile_to_txt(name, about, activity, experience, education, projects, skills, link_url, page_number, cv_number)
                    cv_number += 1
                    driver.back()  # 이전 페이지로 돌아가기
                    time.sleep(2)  # 페이지가 완전히 로드될 때까지 대기
                else:
                    print(f"Profile {link_url} already visited")
            except StaleElementReferenceException:
                print("StaleElementReferenceException occurred. Trying again...")
                collect_profiles_on_page(driver, cv_number)
                break
    else:
        print("No profile links found on this page")


cv_number = 1  # 프로필 번호 초기화
for page_number in range(1, 6):  
    driver.get(f"https://www.linkedin.com/search/results/people/?keywords=RLHF&page={page_number}")  # 페이지로 이동
    time.sleep(5)  # 페이지 로딩 대기

    collect_profiles_on_page(driver, cv_number)

    # 페이지 이동 확인
    if page_number % 20 == 0:  # 한 페이지에서 일정 수의 프로필을 수집한 후 다음 페이지로 이동
        next_button_xpath = f'/html/body/div[4]/div[3]/div[2]/div/div[1]/main/div/div/div[4]/div/div/ul/li[{(page_number // 20) + 2}]/button'
        next_button = driver.find_element(By.XPATH, next_button_xpath)
        next_button.click()
        time.sleep(5)  # 페이지 이동 대기


NameError: name 'StaleElementReferenceException' is not defined

In [10]:
cv_number = 1  # 프로필 번호 초기화
profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
if profile_links:
    for i in range(len(profile_links)):
        try:
            profile_link = profile_links[i].get_attribute('href')
            driver.get(profile_link)
            time.sleep(5)  # 페이지가 로드될 때까지 충분한 시간 대기
            get_text(driver, cv_number)
            cv_number += 1  # 프로필 번호 증가
            if i < len(profile_links) - 1:
                driver.back()  # 이전 페이지로 돌아가기
                time.sleep(2)  # 페이지가 완전히 로드될 때까지 대기
                profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
        except Exception as e:
            print(f"Error processing profile: {e}")
else:
    print("No profile links found")



: 

In [147]:
# 페이지의 첫 번째 프로필 수집
#page_number = 2
cv_number = 1  # 프로필 번호 초기화

profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
if profile_links:
    profile_link = profile_links[4].get_attribute('href')
    driver.get(profile_link)
    time.sleep(5)  # 페이지가 로드될 때까지 충분한 시간 대기
    get_text(driver, cv_number)
    #cv_number += 1  # 프로필 번호 증가
else:
    print("No profile links found")

In [40]:
driver.back()

In [41]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") # 페이지 하단으로 스크롤
time.sleep(2)

In [43]:
#두번째 페이지
# /html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[6]/div/div/button[2](next botton)
# /html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[6]/div/div/ul/li[2]/button(second botton)
# /html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[4]/div/div/ul/li[2]/button
next_button = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[4]/div/div/ul/li[2]/button')
next_button.click()
time.sleep(5)  # 다음 페이지가 로드될 때까지 기다리기

In [81]:
# 페이지의 첫 번째 프로필 수집
page_number = 2
cv_number = 9  # 프로필 번호 초기화

profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
if profile_links:
    profile_link = profile_links[16].get_attribute('href')
    driver.get(profile_link)
    time.sleep(5)  # 페이지가 로드될 때까지 충분한 시간 대기
    get_text(driver, page_number, cv_number)
    #cv_number += 1  # 프로필 번호 증가
else:
    print("No profile links found")

In [82]:
driver.back()

In [70]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") # 페이지 하단으로 스크롤
time.sleep(2)

In [83]:
#세번째 페이지 버튼
# /html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/div[2]/div/ul/li[3]/button
next_button = driver.find_element(By.XPATH, '/html/body/div[4]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/div[2]/div/ul/li[3]/button')
next_button.click()
time.sleep(5)  # 다음 페이지가 로드될 때까지 기다리기

In [144]:
from selenium.webdriver.common.keys import Keys

# 페이지의 첫 번째 프로필 수집
page_number = 1
cv_number = 6  # 프로필 번호 초기화

profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
if profile_links:
    profile_link = profile_links[10].get_attribute('href')
    driver.get(profile_link)
    time.sleep(5)  # 페이지가 로드될 때까지 충분한 시간 대기
    get_text(driver, page_number, cv_number)
    #cv_number += 1  # 프로필 번호 증가
else:
    print("No profile links found")


In [145]:
driver.back()

In [33]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") # 페이지 하단으로 스크롤
time.sleep(2)

In [35]:
# 네번째 페이지 버튼
next_button = driver.find_element(By.XPATH, '/html/body/div[4]/div[3]/div[2]/div/div[1]/main/div/div/div[4]/div/div/ul/li[4]/button')
next_button.click()
time.sleep(5)  # 다음 페이지가 로드될 때까지 기다리기

In [ ]:
from selenium.webdriver.common.keys import Keys

page_number = 4

# 페이지의 첫 번째 프로필 수집
profile_links = driver.find_elements(By.XPATH, "//a[contains(@href,'/in/')]")
if profile_links:
    first_profile_link = profile_links[0].get_attribute('href')
    driver.get(first_profile_link)
    time.sleep(5)  # 페이지가 로드될 때까지 충분한 시간 대기

    # 페이지의 끝까지 스크롤하여 모든 프로필 로드
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 스크롤 후 페이지 로딩을 기다리는 시간
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    get_text(driver, page_number)
else:
    print("No profile links found")

In [ ]:
driver.back()

In [ ]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") # 페이지 하단으로 스크롤
time.sleep(2)

In [ ]:
# 다섯번째 페이지 버튼
next_button = driver.find_element(By.XPATH, '/html/body/div[4]/div[3]/div[2]/div/div[1]/main/div/div/div[4]/div/div/ul/li[5]/button')
next_button.click()
time.sleep(5)  # 다음 페이지가 로드될 때까지 기다리기

In [ ]:
# 여섯번째 페이지 버튼
next_button = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/div[2]/div/ul/li[6]/button')
next_button.click()
time.sleep(5)  # 다음 페이지가 로드될 때까지 기다리기